In [236]:
import pandas as pd
import re
import spacy
import json
from random import randint
from spacy.lang.en.stop_words import STOP_WORDS
from typing import Any, Optional, NamedTuple
from string import punctuation
from datetime import date
from itertools import combinations_with_replacement
from collections import Counter, defaultdict
from calendar import month_name, month_abbr

In [237]:
nlp = spacy.load("en_core_web_lg")

In [238]:
d = pd.read_csv('data/transaction_texts.csv')

In [239]:
class Transaction:
    
    def __init__(self, account_category, account_name, description):
        
        self.account_category: Optional[str] = account_category
        self.account_name: Optional[str] = account_name
        self.description: Optional[str] = description
            
        self.docs = defaultdict()
        self.labels = defaultdict()
    
    def __str__(self):
        return f'{"acc.cat:":<10}{self.account_category}\n{"acc.name:":<10}{self.account_name}\n{"desc:":<10}{self.description}'
    
    def similarity(self, another_transaction):
        
        sim_acc_cat = self.docs['account_category'].similarity(another_transaction.docs['account_category'])
        sim_acc_name = self.docs['account_name'].similarity(another_transaction.docs['account_name'])
        sim_desc = self.docs['description'].similarity(another_transaction.docs['description'])
        
        print(f'similarity: acc.cat: {sim_acc_cat:.4f} acc.name: {sim_acc_name:.4f} description: {sim_desc:.4f}')
        
        return (sim_acc_cat, sim_acc_name, sim_desc)

In [256]:
class Parser:
    
    def __init__(self):
        
        self.abbrs = json.load(open('data/acdic.json'))
        self.syns = {'revenue': set('revenue unbilled earnings gain income incoming proceeds profit return yield unrealized'.split()),
                     'expense': set('expense charge expenditure obligation spending spend overhead surcharge cost'.split()),
                     'accrue': set('accrue accumulate amass collect gather aggregate hoard'.split()),
                     'consolidation': set('consolidation merger strengthening unification amalgamation'.split()),
                     'adjustment': set('adjustment adjust adj alteration correction modification readjustment'.split()),
                     'recurring': set('regular periodic monthly weekly fortnightly yearly repeat routine recurring'.split()),
                     'defer': set('adjourn delay postpone suspend'.split()),
                     'subscription': set('subscription'.split()),
                     'payroll': set('payroll salary wages pay remuneration paycheck earning'.split()),
                     'liability': set('liability debt burden obligation owing uninvoiced'.split()),
                     'rent': set('rent lease rental'.split()),
                     'depreciation': set('depreciation devaluation markdown deflation'.split()),
                     'tax': set('tax duty levy toll tariff excise'.split()),
                     'maintenance': set('maintenance repair'.split()),
                     'entertainment': set('entertainment recreation party'.split()),
                     'prepaid': set('prepaid'.split()),
                     'month': set(map(lambda x: x.lower(), set(month_name[1:]) | set(month_abbr[1:]))),
                     'food': set('food meal dinner lunch restaurant cafe brunch breakfast catering wine beer drinks'.split()),
                     'reverse': set('reverse back return inverse converse'.split())}
        
        for k in self.syns:
            syns_upd = set()
            for w in nlp(' '.join(self.syns[k])):
                syns_upd.add(w.lemma_.lower())
            self.syns[k] = syns_upd
    
    def run(self, st):
        
        _labs = defaultdict(set) 
        _doc = None
        
        if (not isinstance(st, str)) or (not st.strip()):
            return (_doc, _labs)
        
        _labs['dates'] = self.find_dates(st)
        
        st = st.lower().translate({ord(sep): '' for sep in './'})
        
        st = st.translate(str.maketrans({_: ' ' for _ in punctuation}))
        
        # unfold abbreviations
        st = ' '.join([self.abbrs.get(w, w).lower().replace(',','') for w in st.split()])
        
        # remove numbers
        st = ' '.join([w for w in st.split() if w.isalpha()])
        
        # remove stopwords
        st = ' '.join([w for w in st.split() if w not in STOP_WORDS])
        
        # replace multiple consecutive white spaces with a single one
        st = re.sub(r'\s+', ' ', st).strip()
        
        if st:
            _doc = nlp(st)
        
            for what in self.syns:
                for w in _doc:
                    if w.lemma_ in self.syns[what]:
                        _labs['labels'].add(what)
            
        return (_doc, _labs)
    
    def find_dates(self, st):
        
        plausible = dict(day=set(range(1,32)), 
                         month=set(range(1,13)), 
                         year=set(range(19,23)) | set(range(2019, 2023)))
        
        _dates = set()
        
        # try to create a set of found dates using a single - as separator 
        try:
            _dates |= {re.sub(r'[-\/.]+','-', d) for d in re.findall(r'\b\d{1,2}[-\/.]+\d{1,2}[-\/.]+\d{4}\b|'
                                                                     r'\b\d{4}[-\/.]+\d{1,2}[-\/.]+\d{1,2}\b|'
                                                                     r'\b\d{1,2}[-\/.]+\d{1,2}[-\/.]+\d{1,2}\b', st)}
        except:
            pass
        
        try:
            _dates |= {d + '-' + str(list(month_abbr).index(m.title())) + '-' + '20' + y 
                                  for d, m, y in re.findall(r'\b(\d{1,2})(' + '|'.join([m for m in month_abbr[1:]]) + r')(\d{2}\b)', st, flags=re.IGNORECASE)}
        except:
            pass
        
        if not _dates:
            return None
        
        # dates will be gathered in this set if they are valid
        dates = []
        
        for _date in _dates:
            
            _date_parts = _date.split('-')
            
            position_cands = defaultdict(set)
            
            for i, p in enumerate(_date_parts):
                for q in 'year month day'.split():
                    if int(p) in plausible[q]:
                        position_cands[i].add(q)
            
            if set(position_cands) != set(range(len(_date_parts))):
                continue
            
            for p0 in position_cands[0]:
                for p1 in position_cands[1] - {p0}:
                    for p2 in position_cands[2] - {p0} - {p1}:
                        
                        date_as_dict = {p0: int(_date_parts[0]), p1: int(_date_parts[1]), p2: int(_date_parts[2])}
                        
                        # make sure year is presented as 20YY
                        for _ in date_as_dict:
                            if (_ == 'year') and (len(str(date_as_dict[_])) == 2):
                                date_as_dict.update({_: int('20'+str(date_as_dict[_]))})
                                
                        dates.append(date(**date_as_dict))
        
        

        return dates   

In [257]:
p = Parser()

In [258]:
p.run(' ewfefe 2141x !@@@!2 2021--12-4 a.B 3/1452-1-23m  1FEb24  1.1.22 rev accrued maintenance&repairs -- reversed')

(ewfefe ab revenue accrued maintenance repairs reversed,
 defaultdict(set,
             {'dates': [datetime.date(2022, 1, 1),
               datetime.date(2022, 1, 1),
               datetime.date(2021, 4, 12),
               datetime.date(2021, 12, 4)],
              'labels': {'accrue', 'maintenance', 'revenue', 'reverse'}}))

In [243]:
t2 = Transaction(**d.iloc[randint(0, len(d)-1)])
t2.docs['account_category'], t2.labels['account_category']= p.run(t2.account_category)
t2.docs['account_name'], t2.labels['account_name']= p.run(t2.account_name)
t2.docs['description'], t2.labels['description']= p.run(t2.description)

In [249]:
print(t2.labels)

defaultdict(None, {'account_category': defaultdict(<class 'set'>, {'dates': None, 'labels': {'revenue'}}), 'account_name': defaultdict(<class 'set'>, {'dates': None, 'labels': {'subscription', 'payroll'}}), 'description': defaultdict(<class 'set'>, {'dates': None, 'labels': {'consolidation'}})})


In [245]:
t1.similarity(t2)

similarity: acc.cat: 0.6580 acc.name: 0.5385 description: 0.3652


(0.6579922448636794, 0.5385099011262915, 0.36522501795194706)

In [246]:
t2.labels

defaultdict(None,
            {'account_category': defaultdict(set,
                         {'dates': None, 'labels': {'revenue'}}),
             'account_name': defaultdict(set,
                         {'dates': None,
                          'labels': {'payroll', 'subscription'}}),
             'description': defaultdict(set,
                         {'dates': None, 'labels': {'consolidation'}})})

In [248]:
d.head()

,account_category,account_name,description
0,Revenue - Subscriptions,Revenue-Software Subscription (On Premise)-Unb...,ACP ASC 605 Reversals - 05/31/2018
1,Deferred Revenue - Current,AR - Unbilled - Subscription,Consolidation
2,Accrued Taxes,Payroll-Worker's Comp Payable,1197
3,Fringe,Employer SSec,"Reversed -- JOHNSON, CHERYL LYNN"
4,Cash and Cash Equivalents,Chase Operating,1.00E+11


In [261]:
all_trans = []

for r in d.iterrows():
    
    t2 = Transaction(**r[1][['account_name', 'account_category','description']])
    t2.docs['account_category'], t2.labels['account_category']= p.run(t2.account_category)
    t2.docs['account_name'], t2.labels['account_name']= p.run(t2.account_name)
    t2.docs['description'], t2.labels['description']= p.run(t2.description)
    
    for k in t2.labels:
        if 'subscription' in t2.labels[k]['labels']:
            all_trans.append(r[1])
            print(len(all_trans))
            
    if len(all_trans) == 100:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [262]:
all_trans

[account_category                              Revenue - Subscriptions
 account_name        Revenue-Software Subscription (On Premise)-Unb...
 description                        ACP ASC 605 Reversals - 05/31/2018
 Name: 0, dtype: object,
 account_category                              Revenue - Subscriptions
 account_name        Revenue-Software Subscription (On Premise)-Unb...
 description                        ACP ASC 605 Reversals - 05/31/2018
 Name: 0, dtype: object,
 account_category      Deferred Revenue - Current
 account_name        AR - Unbilled - Subscription
 description                        Consolidation
 Name: 1, dtype: object,
 account_category      Deferred Revenue - Current
 account_name        AR - Unbilled - Subscription
 description                        Consolidation
 Name: 5, dtype: object,
 account_category                                Other Current Assets
 account_name        Deferred Revenue Unbilled - Subscription - Guest
 description                      